In [1]:
!which python3

/Users/reddddddddd/Documents/code/projects/spotter/venv/bin/python3


In [35]:
!pip3 -q install googlemaps pandas

  DEPRECATION: googlemaps is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [97]:
import pandas as pd
import googlemaps, requests
from dotenv import load_dotenv

load_dotenv()

In [121]:
data = pd.read_csv("./fuel-prices-for-be-assessment.csv")
n = 1

# keep only nth thousand rows
data = data[1000:]

data.head()

,OPIS Truckstop ID,Truckstop Name,Address,City,State,Rack ID,Retail Price
1000,6441,GRAND OIL,"I-75, EXIT 41 & SR-80",London,KY,410,3.499000
1001,6442,CLARK OIL #49,US-84,Laurel,MS,620,2.999000
1002,6448,CIRCLE K #2723392,"I-59, EXIT 97 & US-84",Laurel,MS,620,2.952333
1003,6462,ARCO #471770,US-281 & SR-5,Dunseith,ND,452,3.499000
1004,6468,TA SAYRE TRAVEL CENTER,"I-40 & CEMETARY RD, EXIT 26",Sayre,OK,483,3.064000


In [122]:
data.shape

(7151, 7)

In [123]:
API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")
gmaps = googlemaps.Client(key=API_KEY)


def get_lat_long(street, city, state):
    try:
        full_address = f"{street}, {city}, {state}"
        geocode_result = gmaps.geocode(full_address)
        
        if geocode_result and 'geometry' in geocode_result[0]:
            location = geocode_result[0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            return None, None
    except IndexError:
        # Handles case where geocode_result is an empty list or missing 'geometry'
        return None, None
    except Exception as e:
        # Handles any other general exceptions
        print(f"Error occurred: {e}")
        return None, None

In [94]:
# # using openstreetmap nominatim API

# def get_lat_long(street, city, state):
#     url = f"""https://nominatim.openstreetmap.org/search?street='{street}'&city='{city}'&state='{state}'&country=USA&format=json"""
#     print(url)
#     response = requests.get(url, headers={
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
#     })
#     print(response.headers)

#     return 0,0

In [124]:
for index, row in data.iterrows():
    lat, lng = get_lat_long(row['Address'], row['City'], row['State'])
    data.at[index, 'Latitude'] = lat
    data.at[index, 'Longitude'] = lng


data.head(10)

,OPIS Truckstop ID,Truckstop Name,Address,City,State,Rack ID,Retail Price,Latitude,Longitude
1000,6441,GRAND OIL,"I-75, EXIT 41 & SR-80",London,KY,410,3.499000,37.117240,-84.104903
1001,6442,CLARK OIL #49,US-84,Laurel,MS,620,2.999000,31.686605,-89.125521
1002,6448,CIRCLE K #2723392,"I-59, EXIT 97 & US-84",Laurel,MS,620,2.952333,31.695851,-88.993702
1003,6462,ARCO #471770,US-281 & SR-5,Dunseith,ND,452,3.499000,48.805781,-100.061125
1004,6468,TA SAYRE TRAVEL CENTER,"I-40 & CEMETARY RD, EXIT 26",Sayre,OK,483,3.064000,35.335535,-99.594561
1005,6480,FLYING J #934,"I-84, EXIT 265",La Grande,OR,982,3.535667,45.347892,-118.155323
1006,6492,FLICKS,US-49,Richland,MS,760,2.999000,32.231443,-90.161664
1007,6496,SPEEDY EXPRESS #38,"I-35, EXIT 305",Mathis,TX,625,3.199000,31.967535,-97.119490
1008,6502,SQUAW CREEK TRAVEL PLAZA,"I-29, EXIT 79 & US-159",Forest City,MO,400,3.339000,44.060015,-96.758754
1009,6517,HARDYS #5,"I-80, EXIT 4 4",Wendover,UT,835,3.499000,40.744109,-114.024396


In [125]:
# save to csv with headers
columns = data.columns.tolist()
data.to_csv(f"fuel-prices-for-be-assessment-with-lat-long-from-1000.csv", columns=columns, index=False)
print(f"Saved to fuel-prices-for-be-assessment-with-lat-long-from-1000.csv")

Saved to fuel-prices-for-be-assessment-with-lat-long-from-1000.csv
